In [56]:
import json
import pandas as pd
import re


In [57]:
# load nodes data

nodes = pd.read_csv('nodes.csv')
nodes

with open('dict_blocks.json', 'r') as json_file:
    dict_blocks = json.load(json_file)

dict_blocks.keys()

dict_keys(['Package\\Merge and filter\\Alter name pref', 'Package\\Merge and filter\\Determine rows sup', 'Package\\Merge and filter\\Load supplier', 'Package\\Merge and filter\\Match with products', 'Package\\Merge and filter\\Split preferences', 'Package\\Merge and filter\\Union suppliers', 'Package\\Merge and filter\\Write Update sup table'])

In [58]:
nodes

,Unnamed: 0,ID_block_out,ID_block_in,type_block_out,type_block_in
0,0,Package\Merge and filter\Split preferences,Package\Merge and filter\Alter name pref,Microsoft.ConditionalSplit,Microsoft.DerivedColumn
1,1,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers,Microsoft.ConditionalSplit,Microsoft.UnionAll
2,2,Package\Merge and filter\Alter name pref,Package\Merge and filter\Union suppliers,Microsoft.DerivedColumn,Microsoft.UnionAll
3,3,Package\Merge and filter\Match with products,Package\Merge and filter\Split preferences,Microsoft.Lookup,Microsoft.ConditionalSplit
4,4,Package\Merge and filter\Load supplier,Package\Merge and filter\Determine rows sup,Microsoft.SSISODBCSrc,Microsoft.RowCount
5,5,Package\Merge and filter\Determine rows sup,Package\Merge and filter\Match with products,Microsoft.RowCount,Microsoft.Lookup
6,6,Package\Merge and filter\Union suppliers,Package\Merge and filter\Write Update sup table,Microsoft.UnionAll,Microsoft.SSISODBCDst


In [59]:
# sort nodes

def order_df(nodes: pd.DataFrame) -> list[list]:
    
    start_node = nodes[~nodes['ID_block_out'].isin(nodes['ID_block_in'])]
    ordered_rows = []
    current_node = start_node.iloc[0]['ID_block_out']
    
    while True:
        current_row = nodes[nodes['ID_block_out'] == current_node]
        ordered_rows.append(current_row)
        if current_row.empty or not any(nodes['ID_block_out'].isin(current_row['ID_block_in'])):
            break
        current_node = current_row.iloc[0]['ID_block_in']
        
    ordered_df = pd.concat(ordered_rows).reset_index(drop=True)
    list_of_lists = ordered_df[['ID_block_out', 'ID_block_in']].values.tolist()
    return list_of_lists

            
sorted_nodes = order_df(nodes)

sorted_nodes

[['Package\\Merge and filter\\Load supplier',
  'Package\\Merge and filter\\Determine rows sup'],
 ['Package\\Merge and filter\\Determine rows sup',
  'Package\\Merge and filter\\Match with products'],
 ['Package\\Merge and filter\\Match with products',
  'Package\\Merge and filter\\Split preferences'],
 ['Package\\Merge and filter\\Split preferences',
  'Package\\Merge and filter\\Alter name pref'],
 ['Package\\Merge and filter\\Split preferences',
  'Package\\Merge and filter\\Union suppliers'],
 ['Package\\Merge and filter\\Alter name pref',
  'Package\\Merge and filter\\Union suppliers'],
 ['Package\\Merge and filter\\Union suppliers',
  'Package\\Merge and filter\\Write Update sup table']]

In [64]:
# add columns data to sorted nodes
def add_metadata(sorted_nodes:list) -> list:
    sorted_nodes_metadata = []

    for i in sorted_nodes:
        dictionary = {}

        dictionary[i[0]] = dict_blocks[i[0]]
        dictionary[i[1]] = dict_blocks[i[1]]

        sorted_nodes_metadata.append(dictionary)
        
    return sorted_nodes_metadata

sorted_nodes_metadata = add_metadata(sorted_nodes)

# standardize columns input and output

In [65]:
nodes

,Unnamed: 0,ID_block_out,ID_block_in,type_block_out,type_block_in
0,0,Package\Merge and filter\Split preferences,Package\Merge and filter\Alter name pref,Microsoft.ConditionalSplit,Microsoft.DerivedColumn
1,1,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers,Microsoft.ConditionalSplit,Microsoft.UnionAll
2,2,Package\Merge and filter\Alter name pref,Package\Merge and filter\Union suppliers,Microsoft.DerivedColumn,Microsoft.UnionAll
3,3,Package\Merge and filter\Match with products,Package\Merge and filter\Split preferences,Microsoft.Lookup,Microsoft.ConditionalSplit
4,4,Package\Merge and filter\Load supplier,Package\Merge and filter\Determine rows sup,Microsoft.SSISODBCSrc,Microsoft.RowCount
5,5,Package\Merge and filter\Determine rows sup,Package\Merge and filter\Match with products,Microsoft.RowCount,Microsoft.Lookup
6,6,Package\Merge and filter\Union suppliers,Package\Merge and filter\Write Update sup table,Microsoft.UnionAll,Microsoft.SSISODBCDst


In [66]:
def extract_column(text: str) -> str:
    match = re.search(r'\[([^\]]*)\]', text)
    return match.group(1) if match else ''




In [67]:
columns_input = []
columns_output = []

seen_nodes = []

previous_node = None

for nodes_pair in sorted_nodes_metadata:


    for i, node in enumerate(nodes_pair):
        #print(node)

        # define previous node
        if i == 0:
            previous_node = node

        # define the type of node
        if i == 1:
            type = nodes.loc[nodes['ID_block_in'] == node, 'type_block_in'].iloc[0]#.item()
        if i == 0:
            type = nodes.loc[nodes['ID_block_out'] == node, 'type_block_out'].iloc[0]#.item()


        if previous_node:
            # if node pair was already seen then skip
            if [previous_node, node] in seen_nodes:
                continue
            # if the previous node is the same as the current node and the node type is not source then skip
            if previous_node == node and type != 'Microsoft.SSISODBCSrc':
                continue
            print([previous_node, node])
            seen_nodes.append([previous_node, node])

        print(type)


        # process source node
        if type == 'Microsoft.SSISODBCSrc':
            columns=[d['Column_name'] for d in nodes_pair[node]]

            columns_in = [d['Column_input'] for d in nodes_pair[node]]
            columns_out = [node +'[' + d['Column_name']  +']' for d in nodes_pair[node]]

            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))
            columns_input += columns_in
            columns_output += columns_out

        # process lookup node
        if type == 'Microsoft.Lookup':

            columns_in =[d['Column_lookup'] for d in nodes_pair[node]['merged_columns']] + [previous_node + '[' +column+']' for column in columns]
            columns_out=[node +'[' +d['Column_name']+']' for d in nodes_pair[node]['merged_columns']]+ [node + '[' +column+']' for column in columns]

            #columns_in = set([previous_node + '[' +column+']' for column in columns] + [d['Column_lookup'] for d in nodes_pair[node]['merged_columns']])
            #columns_out= set([node + '[' +column+']' for column in columns] + [node + '[' +d['Column_name']+']' for d in nodes_pair[node]['merged_columns']])

            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))

            columns_input +=columns_in
            columns_output +=columns_out
            columns += [d['Column_name'] for d in nodes_pair[node]['merged_columns']]

        # process derivedcolumn node
        if type == 'Microsoft.DerivedColumn':
            
            expression=[d['expression'] for d in nodes_pair[node]]

            columns_in =[previous_node + '[' + column+']' for column in columns]
            columns_out=[node +'[' +column+']' for column in columns]

            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))
            print(expression)

            columns_input +=columns_in
            columns_output +=columns_out


        if type == 'Microsoft.RowCount':

            columns_in =[previous_node + '[' + column+']' for column in columns]
            columns_out=[node +'[' +column+']' for column in columns]

            variable =nodes_pair[node]

            print(variable)
            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))

            columns_input +=columns_in
            columns_output +=columns_out
            
        if type == 'Microsoft.ConditionalSplit':
            columns_in=[previous_node + '[' + column+']' for column in columns]
            columns_out=[node +'[' +column+']' for column in columns]

            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))

            columns_input +=columns_in
            columns_output +=columns_out          

        if type == 'Microsoft.UnionAll':
            columns_in=[d['Column_input'] for d in nodes_pair[node] if previous_node in d['Column_input']] # add the column that are mentioned in the previous node, otherwise the same column is added two times
            columns_out=[d['Column_name'] for d in nodes_pair[node]]

            columns_out = [node +'[' +n  +']' for n in set(columns_out)]

            columns_in = sorted(columns_in, key=extract_column)
            columns_out = sorted(columns_out, key=extract_column)


            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))

            columns_input +=columns_in
            columns_output +=columns_out
            
        if type == 'Microsoft.SSISODBCDst':
            columns_in =[previous_node + '[' + d['Column_name']+']' for d in nodes_pair[node]]
            columns_out=[node +'[' +d['Column_name']+']' for d in nodes_pair[node]]

            print(columns_in)
            print(columns_out)
            print(len(columns_in))
            print(len(columns_out))
            columns_input +=columns_in
            columns_output +=columns_out

        print(previous_node)
        print()



['Package\\Merge and filter\\Load supplier', 'Package\\Merge and filter\\Load supplier']
Microsoft.SSISODBCSrc
['"dbo"."Suppliers_Extract"[ValidFrom]', '"dbo"."Suppliers_Extract"[ValidTo]', '"dbo"."Suppliers_Extract"[Active]', '"dbo"."Suppliers_Extract"[Country]', '"dbo"."Suppliers_Extract"[SupplierID]', '"dbo"."Suppliers_Extract"[CompanyName]', '"dbo"."Suppliers_Extract"[Preferred]', '"dbo"."Suppliers_Extract"[Discount]', '"dbo"."Suppliers_Extract"[Address]', '"dbo"."Suppliers_Extract"[Region]', '"dbo"."Suppliers_Extract"[Fax]', '"dbo"."Suppliers_Extract"[Phone]', '"dbo"."Suppliers_Extract"[City]', '"dbo"."Suppliers_Extract"[ContactName]', '"dbo"."Suppliers_Extract"[ContactTitle]', '"dbo"."Suppliers_Extract"[PostalCode]']
['Package\\Merge and filter\\Load supplier[ValidFrom]', 'Package\\Merge and filter\\Load supplier[ValidTo]', 'Package\\Merge and filter\\Load supplier[Active]', 'Package\\Merge and filter\\Load supplier[Country]', 'Package\\Merge and filter\\Load supplier[SupplierID]

In [ ]:
(columns_input).__len__()
(columns_output).__len__()

lineages = {'column_in': columns_input, 'column_out': columns_output}

lineages = pd.DataFrame(lineages)
lineages['col_name_in'] = lineages['column_in'].str.extract(r'\[([^\]]*)\]')
lineages['col_name_out'] = lineages['column_out'].str.extract(r'\[([^\]]*)\]')

lineages['node_in'] =lineages['column_in'].str.extract(r'^(.*?)\s*\[.*\]$')
lineages['node_out'] =lineages['column_out'].str.extract(r'^(.*?)\s*\[.*\]$')


lineages[-50:]

,column_in,column_out,col_name_in,col_name_out,node_in,node_out
84,Package\Merge and filter\Split preferences[Add...,Package\Merge and filter\Union suppliers[Address],Address,Address,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
85,Package\Merge and filter\Split preferences[City],Package\Merge and filter\Union suppliers[City],City,City,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
86,Package\Merge and filter\Split preferences[Com...,Package\Merge and filter\Union suppliers[Compa...,CompanyName,CompanyName,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
87,Package\Merge and filter\Split preferences[Con...,Package\Merge and filter\Union suppliers[Conta...,ContactName,ContactName,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
88,Package\Merge and filter\Split preferences[Con...,Package\Merge and filter\Union suppliers[Conta...,ContactTitle,ContactTitle,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
89,Package\Merge and filter\Split preferences[Cou...,Package\Merge and filter\Union suppliers[Country],Country,Country,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
90,Package\Merge and filter\Split preferences[Dis...,Package\Merge and filter\Union suppliers[Disco...,Discount,Discount,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
91,Package\Merge and filter\Split preferences[Fax],Package\Merge and filter\Union suppliers[Fax],Fax,Fax,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
92,Package\Merge and filter\Split preferences[Phone],Package\Merge and filter\Union suppliers[Phone],Phone,Phone,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
93,Package\Merge and filter\Split preferences[Pos...,Package\Merge and filter\Union suppliers[Posta...,PostalCode,PostalCode,Package\Merge and filter\Split preferences,Package\Merge and filter\Union suppliers
